In [8]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import GroverOperator, MCMT, ZGate
import math

In [9]:

# def grover_oracle(marked_states):
#     if not isinstance(marked_states, list):
#         marked_states = [marked_states]
#     num_qubits = len(marked_states[0])

#     qc = QuantumCircuit(num_qubits)
#     for target in marked_states:
#         rev_target = target[::-1]

#         zero_inds = [ind for ind in range(num_qubits) if rev_target.startswith("0", ind)]

#         qc.x(zero_inds)
#         qc.compose(MCMT(ZGate(), num_qubits - 1, 1), inplace=True)
#         qc.x(zero_inds)
#     return qc

# marked_states = ["011", "100"]

# oracle = grover_oracle(marked_states)
# oracle.draw(output="mpl", style="iqp")

In [10]:
def grover_oracle_3sat(clauses, num_vars):
    num_clauses = len(clauses)
    total_qubits = num_vars + num_clauses + 1
    qc = QuantumCircuit(total_qubits)
    
    # Label registers: assignment qubits = 0 ... num_vars-1,
    # clause ancillas = num_vars ... num_vars+num_clauses-1,
    # phase qubit = total_qubits-1.
    assignment = list(range(num_vars))
    clause_ancillas = list(range(num_vars, num_vars + num_clauses))
    phase_qubit = total_qubits - 1

    # Prepare phase qubit in |–> state.
    qc.h(phase_qubit)
    qc.x(phase_qubit)
    
    # For each clause, mark a clause-ancilla if the clause is false.
    # A clause (l1 OR l2 OR l3) is false only if all three literals are false.
    # For each literal:
    #   - If positive, the qubit should be True (|1>), so we flip (X) the qubit to detect |0>.
    #   - If negative, no change is needed since |1> already means false.
    for i, clause in enumerate(clauses):
        for literal in clause:
            var_index = abs(literal) - 1  # convert 1-indexed to 0-indexed.
            if literal > 0:
                qc.x(assignment[var_index])
        
        # If all three (modified) assignment qubits are 1, then the clause is false.
        qc.mcx([abs(lit)-1 for lit in clause], clause_ancillas[i])
        
        # Undo the X gates applied for positive literals.
        for literal in clause:
            var_index = abs(literal) - 1
            if literal > 0:
                qc.x(assignment[var_index])
                
    qc.barrier()
    # Invert clause ancillas so that a 1 indicates the clause is satisfied.
    for i in range(num_clauses):
        qc.x(clause_ancillas[i])
        
    qc.barrier()
    # When all inverted clause ancillas are 1, every clause is satisfied.
    # Use them as controls to flip the phase qubit.
    qc.mcx(clause_ancillas, phase_qubit)
    
    qc.barrier()
    # Undo the inversion of clause ancillas.
    for i in range(num_clauses):
        qc.x(clause_ancillas[i])
        
    # Uncompute the clause ancillas.
    for i, clause in reversed(list(enumerate(clauses))):
        for literal in clause:
            var_index = abs(literal) - 1
            if literal > 0:
                qc.x(assignment[var_index])
        qc.mcx([abs(lit)-1 for lit in clause], clause_ancillas[i])
        for literal in clause:
            var_index = abs(literal) - 1
            if literal > 0:
                qc.x(assignment[var_index])
        qc.barrier()
        
    # Restore the phase qubit to the computational basis.
    qc.x(phase_qubit)
    qc.h(phase_qubit)
    qc.barrier()
    
    return qc

# Example usage:
# Suppose we have a 3-SAT formula with 3 variables and 2 clauses:
# Clause 1: (x1 OR ¬x2 OR x3) represented as [1, -2, 3]
# Clause 2: (¬x1 OR x2 OR x3) represented as [-1, 2, 3]
clauses = [
    [1, -2, -3],  # forces x1=1: if x1 is 0 then clause is false because -2 and -3 require x2=0, x3=0
    [-1, 2, -3],  # forces x2=1
    [-1, -2, 3]   # forces x3=1
]
num_vars = 3

oracle = grover_oracle_3sat(clauses, num_vars)
# oracle.draw(output="mpl", style="iqp")

In [11]:
grover_op = GroverOperator(oracle)
# grover_op.decompose().draw(output="mpl", style="iqp")

In [12]:
N = grover_op.num_qubits
optimal_num_iterations = math.floor(math.pi / (4 * math.asin(math.sqrt(1/2**N))))
print("Optimal iterations:", optimal_num_iterations)

Optimal iterations: 8


In [13]:
qc = QuantumCircuit(N, num_vars)
qc.h(range(N))
qc.compose(grover_op.power(optimal_num_iterations), inplace=True)
# Measure only the assignment qubits (first num_vars qubits).
qc.measure(list(range(num_vars)), list(range(num_vars)))
# qc.draw(output="mpl", style="iqp")

In [14]:
simulator = AerSimulator()
qc_compiled = transpile(qc, simulator)
job = simulator.run(qc_compiled)
result = job.result()
counts = result.get_counts(qc_compiled)
print(counts)
# plot_histogram(counts)

{'001': 116, '110': 123, '101': 133, '100': 122, '000': 119, '010': 142, '011': 141, '111': 128}
